### Importing libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#!pip install geopy 
#!pip install Nominatim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Nominatim generates longitude, latitude of most of the city,state around the world
from  geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=None)

/usr/local/lib/python3.8/dist-packages/geopy/geocoders/osm.py:132: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  warnings.warn(


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Data science/Data science/anpa/Anpa_members_map/ANPA data.csv')
data.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership
2,NaN,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership


In [ ]:
data['City']=data['City'].fillna('') # replacing nan with " " was helpful when trying to generate location in next step
data.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership


In [ ]:
data.loc[data['Country'] != 'US','Country']= 'US'

In [ ]:
# combine city, state, and country to create new columns address
data['address']=data['City'].astype(str)+','+data['State'].astype(str)+ ','+data['Country'].astype(str)
data.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership,"Worcester,MA,US"
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership,"Pittsburgh,PA,US"
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-,",Illinois,US"
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership,"Lubbock,Texas,US"
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership,"Kansas City,Missouri,US"


In [ ]:
data.shape

(379, 7)

In [ ]:
data.loc[(data['State'].isin(['TN', 'MI', 'California', 'NC', 'Michigan', 'Florida', 'Illinois', 'TX'])) & (data['Level']=='-'),'Level']="Lifetime Membership"
data.loc[data["Level"]=='-',"Level"]="Student Membership"
data['Level'].value_counts()

Lifetime Membership        117
First Time Student         117
Student Membership         108
Regular Membership          36
General Term Membership      1
Name: Level, dtype: int64

In [ ]:
## Drop duplicate addresses
data = data.drop_duplicates(subset= ["City","Country","State"], keep = 'first')

In [ ]:
# getting new column with longitude and latitude based off address 
data['Location']=data['address'].apply(lambda x:geolocator.geocode(x))
data.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership,"Worcester,MA,US","(Worcester, Worcester County, Massachusetts, U..."
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership,"Pittsburgh,PA,US","(Pittsburgh, Allegheny County, Pennsylvania, U..."
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",Lifetime Membership,",Illinois,US","(Illinois, South Denley Drive, Dallas, Dallas ..."
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership,"Lubbock,Texas,US","(Lubbock, Lubbock County, Texas, United States..."
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership,"Kansas City,Missouri,US","(Kansas City, Jackson County, Missouri, United..."


In [ ]:
data['Level'].value_counts()

Lifetime Membership    97
First Time Student     58
Student Membership     57
Regular Membership     28
Name: Level, dtype: int64

In [ ]:
# It seems the location columns has returned null for 10 entries.
data.isnull().sum()

City                       0
Country                    0
State                      1
Current Affiliation        0
Field of Study/Interest    1
Level                      0
address                    0
Location                   0
dtype: int64

In [ ]:
data[data['Location'].isnull()]
# There are mismatch in the city, country, and state. For example,  the code didn't find the valid address for "Bedford, Texas,NP". 
# We have 10 such entries where location is returned as None. 
#I will discard these 10 entries for now and use only rest of the valid entries for creating map.

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location


In [ ]:
# selecting only valid entries
data1=data[data['Location'].notnull()]
data1.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership,"Worcester,MA,US","(Worcester, Worcester County, Massachusetts, U..."
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership,"Pittsburgh,PA,US","(Pittsburgh, Allegheny County, Pennsylvania, U..."
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",Lifetime Membership,",Illinois,US","(Illinois, South Denley Drive, Dallas, Dallas ..."
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership,"Lubbock,Texas,US","(Lubbock, Lubbock County, Texas, United States..."
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership,"Kansas City,Missouri,US","(Kansas City, Jackson County, Missouri, United..."


In [ ]:
# creating two new columns for latitude and longitude from the location columns
data1['Latitude']=data1['Location'].apply(lambda x:x.latitude)
data1['Longitude']=data1['Location'].apply(lambda x:x.longitude)
data1.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location,Latitude,Longitude
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership,"Worcester,MA,US","(Worcester, Worcester County, Massachusetts, U...",42.262562,-71.801888
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership,"Pittsburgh,PA,US","(Pittsburgh, Allegheny County, Pennsylvania, U...",40.441694,-79.990086
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",Lifetime Membership,",Illinois,US","(Illinois, South Denley Drive, Dallas, Dallas ...",32.723180,-96.805191
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership,"Lubbock,Texas,US","(Lubbock, Lubbock County, Texas, United States...",33.563521,-101.879336
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership,"Kansas City,Missouri,US","(Kansas City, Jackson County, Missouri, United...",39.100105,-94.578142


### Map using Folium

In [ ]:
import folium

In [ ]:
world_map=folium.Map(location=[23.63, -102.35],tiles='Stamen Terrain' ,zoom_start=4)
world_map

In [ ]:
data1.columns

Index(['City', 'Country', 'State', 'Current Affiliation',
       'Field of Study/Interest', 'Level', 'address', 'Location', 'Latitude',
       'Longitude'],
      dtype='object')

In [ ]:
data1.Level.unique() # I want to use as Level feature as color code in the map. Looking at the unique values.

array(['Lifetime Membership', 'Student Membership', 'Regular Membership',
       'First Time Student'], dtype=object)

In [ ]:
#assigning color to each features 
color_map={'Lifetime Membership':'red','Student Membership':'green',
       'Regular Membership':'blue', 'First Time Student':'orange'}

In [ ]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for index,record in data1.iterrows():
  folium.Marker(location=[record[8],
  record[9]],
  popup=record[5],
  icon=folium.Icon(color=color_map[record[5]],
  icon='')
  ).add_to(incidents)
# add incidents to map
world_map.add_child(incidents)

In [ ]:
# Creating legends
import branca

legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 10px;
    left: 20px;
    width: 250px;
    height: 200px;
    z-index:9999;
    font-size:14px;
    font-weight:bold;
    background-color:#B0C4DE;
    opacity: .85;
    ">
    <p><a style="color:#FF0000 ;font-size:150%;margin-left:20px;">&marker;</a>&emsp;Lifetime Membership</p>
    <p><a style="color:#32CD32;font-size:150%;margin-left:20px;">&marker;</a>&emsp;Student Membership</p>
    <p><a style="color:#00BFFF;font-size:150%;margin-left:20px;">&marker;</a>&emsp;Regular Membership</p>
    <p><a style="color:#FFA500;font-size:150%;margin-left:20px;">&marker;</a>&emsp;First Time Student</p>
</div>
{% endmacro %}
'''
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
folium.LayerControl().add_to(world_map)
world_map.get_root().add_child(legend)
world_map

In [ ]:
world_map.save('Anpa_members.html') # saving html for interactive maps.